Deliverable 2: Create a Customer Travel Destinations Map

In [1]:
import pandas as pd
import requests
import gmaps

from config import g_key
gmaps.configure(api_key=g_key)

In [2]:
city_data_df = pd.read_csv("weatherpy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Current Description,Humidity,Cloudiness,Wind Speed,Country
0,0,Castro,-24.7911,-50.0119,66.56,overcast clouds,75,93,5.68,BR
1,1,Beisfjord,68.3751,17.5988,37.45,light snow,90,100,4.47,NO
2,2,Butaritari,3.0707,172.7902,80.51,light rain,82,94,12.50,KI
3,3,Albany,42.6001,-73.9662,59.88,scattered clouds,28,25,1.99,US
4,4,Rawlins,41.7911,-107.2387,43.36,broken clouds,52,75,11.50,US


Prompt the user to enter minimum and maximum temperature criteria.

In [6]:
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

In [7]:
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Current Description,Humidity,Cloudiness,Wind Speed,Country
2,2,Butaritari,3.0707,172.7902,80.51,light rain,82,94,12.50,KI
8,8,Kapaa,22.0752,-159.3190,74.12,scattered clouds,82,40,19.57,US
15,15,Atuona,-9.8000,-139.0333,78.58,light rain,76,39,13.13,PF
16,16,Sao Mateus,-18.7161,-39.8589,84.42,clear sky,53,7,10.07,BR
17,17,Baikunthpur,23.2500,82.5500,89.08,scattered clouds,11,41,6.73,IN
20,20,San Patricio,28.0170,-97.5169,84.29,few clouds,54,17,13.38,US
21,21,Vila Velha,-20.3297,-40.2925,87.75,few clouds,75,20,9.22,BR
22,22,Sancti Spiritus,21.9297,-79.4425,88.95,scattered clouds,37,29,7.11,CU
25,25,Puerto Ayora,-0.7393,-90.3518,80.56,scattered clouds,81,38,9.31,EC
38,38,Rikitea,-23.1203,-134.9692,75.99,light rain,76,56,14.16,PF


Determine if there are any empty rows.

Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

In [8]:
preferred_cities_df.count()

City_ID                247
City                   247
Lat                    247
Lng                    247
Max Temp               247
Current Description    247
Humidity               247
Cloudiness             247
Wind Speed             247
Country                246
dtype: int64

In [9]:
clean_travel_cities = preferred_cities_df.dropna()
clean_travel_cities.head()

,City_ID,City,Lat,Lng,Max Temp,Current Description,Humidity,Cloudiness,Wind Speed,Country
2,2,Butaritari,3.0707,172.7902,80.51,light rain,82,94,12.50,KI
8,8,Kapaa,22.0752,-159.3190,74.12,scattered clouds,82,40,19.57,US
15,15,Atuona,-9.8000,-139.0333,78.58,light rain,76,39,13.13,PF
16,16,Sao Mateus,-18.7161,-39.8589,84.42,clear sky,53,7,10.07,BR
17,17,Baikunthpur,23.2500,82.5500,89.08,scattered clouds,11,41,6.73,IN


In [10]:
clean_travel_cities.count()

City_ID                246
City                   246
Lat                    246
Lng                    246
Max Temp               246
Current Description    246
Humidity               246
Cloudiness             246
Wind Speed             246
Country                246
dtype: int64

Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.

In [12]:
hotel_df = clean_travel_cities[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Butaritari,KI,80.51,light rain,3.0707,172.7902,
8,Kapaa,US,74.12,scattered clouds,22.0752,-159.3190,
15,Atuona,PF,78.58,light rain,-9.8000,-139.0333,
16,Sao Mateus,BR,84.42,clear sky,-18.7161,-39.8589,
17,Baikunthpur,IN,89.08,scattered clouds,23.2500,82.5500,
20,San Patricio,US,84.29,few clouds,28.0170,-97.5169,
21,Vila Velha,BR,87.75,few clouds,-20.3297,-40.2925,
22,Sancti Spiritus,CU,88.95,scattered clouds,21.9297,-79.4425,
25,Puerto Ayora,EC,80.56,scattered clouds,-0.7393,-90.3518,
38,Rikitea,PF,75.99,light rain,-23.1203,-134.9692,


* Set parameters to search for hotels with 5000 meters.

* Iterate through the hotel DataFrame.

* Get latitude and longitude from DataFrame.

* Set up the base URL for the Google Directions API to get JSON data.

* Make request and retrieve the JSON data from the search.

*  Get the first hotel from the results and store the name, if a hotel isn't found skip the city.

In [13]:
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [15]:
for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]

    
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotels = requests.get(base_url, params=params).json()
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [16]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Butaritari,KI,80.51,light rain,3.0707,172.7902,Isles Sunset Lodge
8,Kapaa,US,74.12,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
15,Atuona,PF,78.58,light rain,-9.8000,-139.0333,Villa Enata
16,Sao Mateus,BR,84.42,clear sky,-18.7161,-39.8589,Hotel Rio Norte
17,Baikunthpur,IN,89.08,scattered clouds,23.2500,82.5500,Hotel Shree Annapurna
...,...,...,...,...,...,...,...
650,Lalawigan,PH,79.21,scattered clouds,11.5892,125.4697,Palms Apartment
655,Cozumel,MX,86.95,broken clouds,20.5083,-86.9458,Cozumel Palace® All Inclusive Resort
658,Celestun,MX,86.56,clear sky,20.8667,-90.4000,Posada Lilia
662,Ouegoa,NC,73.99,scattered clouds,-20.3500,164.4333,


Drop the rows where there is no Hotel Name.

In [17]:
nan_value = float("NaN")

In [18]:
hotel_df.replace("", nan_value, inplace=True)

In [19]:
hotel_df.dropna(subset = ["Hotel Name"], inplace=True)

In [20]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Butaritari,KI,80.51,light rain,3.0707,172.7902,Isles Sunset Lodge
8,Kapaa,US,74.12,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
15,Atuona,PF,78.58,light rain,-9.8000,-139.0333,Villa Enata
16,Sao Mateus,BR,84.42,clear sky,-18.7161,-39.8589,Hotel Rio Norte
17,Baikunthpur,IN,89.08,scattered clouds,23.2500,82.5500,Hotel Shree Annapurna


Create the output File (CSV).

Export the City_Data into a csv.

In [21]:
output_data_file = "weatherpy_vacation.csv"

hotel_df.to_csv(output_data_file, index_label="City_ID")

Using the template add city name, the country code, the weather description and maximum temperature for the city.

In [22]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

Get the data from each row and add it to the formatting template and store the data in a list.

In [23]:
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

Get the latitude and longitude from each row and store in a new DataFrame.

In [24]:
locations = hotel_df[["Lat", "Lng"]]

Add a marker layer for each city to the map.

Display the figure.

In [26]:
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))